## ISMD (Inversed Synthesizable Molecular Design) Totorial

This tutorial will proceed as follow:

1. initial setup and data preparation
2. descriptor preparation for forward model (likelihood)
3. forward model (likelihood) preparation
4. proposal mdoel preparation 
5. a complete ismd run

### 1.1 import packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import xenonpy
import onmt
from xenonpy.descriptor import Fingerprints
from xenonpy.inverse.iqspr import GaussianLogLikelihood
from xenonpy.contrib.ismd import ReactionDescriptor
from xenonpy.contrib.ismd import ReactantPool
from xenonpy.contrib.ismd import Reactor

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy

### 1.2 load data

In [2]:
# ground truth data
ground_truth_path = "/home/qiz/data/lab_database/ismd_data/STEREO_id_reactant_product_xlogp_tpsa.csv"
data = pd.read_csv(ground_truth_path)[:10000]
data.head()

reactant_index                                           reactant  \
0    12163.22445                                CCS(=O)(=O)Cl.OCCBr   
1      863.20896                            CC(C)CS(=O)(=O)Cl.OCCCl   
2         249087                     O=[N+]([O-])c1cccc2cnc(Cl)cc12   
3   153658.23440  Cc1cc2c([N+](=O)[O-])cccc2c[n+]1[O-].O=P(Cl)(C...   
4         297070          CCCCC[C@H](O)C=CC1C=CC(=O)C1CC=CCCCC(=O)O   

                                    product  XLogP  TPSA  
0                          CCS(=O)(=O)OCCBr    0.8  51.8  
1                      CC(C)CS(=O)(=O)OCCCl    1.6  51.8  
2                       Nc1cccc2cnc(Cl)cc12    2.4  38.9  
3         Cc1cc2c([N+](=O)[O-])cccc2c(Cl)n1    3.3  58.7  
4  CCCCC[C@H](O)C=CC1CCC(=O)C1CC=CCCCC(=O)O    3.8  74.6

In [3]:
# reactant pool
reactant_pool_path = "/home/qiz/data/lab_database/ismd_data/STEREO_pool.txt"

with open(reactant_pool_path, 'r') as f: 
    reactant_pool = f.read().splitlines()  # len(reactant_pool)=637645

# show the first three elements in the reactant pool
print(reactant_pool[:3])

['O=C(Cl)Oc1ccc(Cc2ccc(C(F)(F)F)cc2)cc1', 'CCc1cc(C2CCN(C(=O)OC(C)(C)C)CC2)ccc1Nc1ncc(C(F)(F)F)c(C#Cc2ccccc2CC(=O)OC)n1', 'CC(NC(=O)OCc1ccccc1)C(C)NC(=O)c1ccccc1O']


In [4]:
# similarity matrix of reactant pool
sim_matrix_path = "/home/qiz/data/lab_database/ismd_data/ZINC_sim_sparse.npz"
reactant_pool_sim = scipy.sparse.load_npz(sim_matrix_path).tocsr()

# show the list of indice whose molecule is similar to the first one in the reactant pool
print(reactant_pool_sim[0,:].nonzero()[1].tolist())

[0, 9850, 11897, 23561, 25594, 28947, 30750, 31361, 44204, 46017, 76945, 118108, 145556, 145734, 164311, 186671, 205326, 207174, 209595, 215653, 218310, 222491, 224002, 232232, 233447, 252758, 274284, 278177, 288659, 291331, 294003, 294172, 300867, 306289, 307663, 331897, 334538, 335455, 343644, 360531, 364663, 365676, 376086, 378821, 412563, 442160, 443411, 452943, 460860, 479253, 487849, 491373, 499241, 500259, 523929, 525478, 528040, 559770, 567735, 568783, 582833, 584542, 586316, 588491, 595094, 599275, 601808, 603887, 617189]


### 2.1 descripter
data is transformed in the following flow:

index of reactant -> smiles of reactant -> smiles of product -> fingerprint of product

In [5]:
# take some samples (index of reactant)
samples = data["reactant_index"][:10].tolist()
print(samples)

['12163.22445', '863.20896', '249087', '153658.23440', '297070', '208421', '412634.601987', '10425.19854', '9361.387984.30667', '50995.305035']


### 2.1.1 index of reactant -> smiles of reactant
Obtain the smiles by ReactantPool module via index

Note: the ReactantPool also used as proposal model in step 4

In [6]:
        pool_obj = ReactantPool(pool_data=reactant_pool, similarity_matrix=reactant_pool_sim, splitter='.')

In [7]:
pool_obj.index2reactant(samples)

['CCS(=O)(=O)Cl.OCCBr',
 'CC(C)CS(=O)(=O)Cl.OCCCl',
 'O=[N+]([O-])c1cccc2cnc(Cl)cc12',
 'Cc1cc2c([N+](=O)[O-])cccc2c[n+]1[O-].O=P(Cl)(Cl)Cl',
 'CCCCC[C@H](O)C=CC1C=CC(=O)C1CC=CCCCC(=O)O',
 'CC(=O)OCC1=C(C(=O)O)N2C(=O)[C@@H](NC(=O)C(OC(C)=O)c3ccccc3)[C@H]2SC1',
 'COc1cccc(C2(CC(Cl)(Cl)Cl)CO2)c1.ClC(Cl)(Cl)CC1(c2ccc(Br)cc2)CO1',
 'COc1cc2ccccc2cc1C(=O)O.O=S(Cl)Cl',
 'CCN(CC)CC.O.O=C(Cl)Oc1ccccc1',
 'CCOC(N)=O.Cc1ccc(N=C=O)cc1N=C=O']

### 2.1.2 smiles of reactant -> fingerprint of product

In [8]:
# build molecular transformer (smiles of reactant -> smiles of product)
reactor_path = "/home/qiz/data/lab_database/models/STEREO_mixed_augm_model_average_20.pt"
ChemicalReactor = Reactor()
ChemicalReactor.BuildReactor(model_list=[reactor_path], max_length=100, n_best=1, gpu=0)

In [9]:
# build fingerprint descriptor (smiles of product -> fingerprint of product)

RDKit_FPs = Fingerprints(featurizers=['ECFP', 'MACCS'], input_type='smiles')

In [10]:
# build reaction descriptor (index of reactant -> fingerprint of product)
# a combination of reactor and fingerprint descripter

RD = ReactionDescriptor(descriptor_calculator=RDKit_FPs,reactor=ChemicalReactor,reactant_pool=pool_obj)

In [11]:
sample_fps = RD.transform(samples)
sample_fps.head(3)

maccs:0  maccs:1  maccs:2  maccs:3  maccs:4  maccs:5  maccs:6  maccs:7  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   

   maccs:8  maccs:9  ...  ecfp3:2038  ecfp3:2039  ecfp3:2040  ecfp3:2041  \
0        0        0  ...           0           0           0           0   
1        0        0  ...           0           0           0           0   
2        0        0  ...           0           0           0           0   

   ecfp3:2042  ecfp3:2043  ecfp3:2044  ecfp3:2045  ecfp3:2046  ecfp3:2047  
0           0           0           0           0           0           0  
1           0           0           0           0           0           0  
2           0           0           0           0           0           0  

[3 rows x 2215 columns]

### 3 Log-likelihood calculator

Compute the log-likelihood given the samples(index of reactant)

In [12]:
# set target
prop = ['XLogP', 'TPSA']
target_range = {'XLogP': (-2, 2), 'TPSA': (0, 25)}

# build Gaussian likelihood calculator and set the target of region of the properties
likelihood_calculator = GaussianLogLikelihood(descriptor=RD, targets = target_range)

In [13]:
%%time

# train forward models inside ismd
likelihood_calculator.fit(data['reactant_index'], data[prop])

RDKit ERROR: [15:07:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 13
RDKit ERROR: 
RDKit ERROR: [15:07:29] Can't kekulize mol.  Unkekulized atoms: 6 13 14 15 22 23 24
RDKit ERROR: 
RDKit ERROR: [15:07:29] Can't kekulize mol.  Unkekulized atoms: 9 10 11 13 14 15 16 18 19
RDKit ERROR: 
RDKit ERROR: [15:07:29] Can't kekulize mol.  Unkekulized atoms: 9 10 11 13 14 15 16 18 19
RDKit ERROR: 
RDKit ERROR: [15:07:29] SMILES Parse Error: unclosed ring for input: 'CC(=O)O[C@H]1[C@]2(O)C[C@]3(O)SS[C@]4(CO)C(=O)N(C)C(=O)N3[C@@H]2C[C@@]13OC(C)C(C)(C)C3=O'
RDKit ERROR: [15:07:29] SMILES Parse Error: unclosed ring for input: 'COc1cccc(C23CCc4[nH]nc(O)c4C2)c1'
RDKit ERROR: [15:07:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 7 8 9 10 12 13
RDKit ERROR: 
RDKit ERROR: [15:07:29] SMILES Parse Error: syntax error while parsing: CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)[C@@H](NC(=O)[C@H](CCC(N)=O)NC(=O)[C@@H]1CCCN1C(=O)[C@H](Cc1ccccc1)NC(=O)[C@@H](NC(=O)OCc1ccccc1)[C@@H](
RDKit ERROR: [15:07:2

CPU times: user 5min, sys: 6.32 s, total: 5min 6s
Wall time: 2min 27s


In [14]:
# predicted properties of samples
property_prediction = likelihood_calculator.predict(samples)
print(property_prediction.head())

   XLogP: mean  XLogP: std  TPSA: mean  TPSA: std
0     3.272444    1.983178   60.023316  40.707506
1     3.136484    1.990339   57.212363  40.886018
2     2.871614    1.992173   63.011102  40.944258
3     3.153963    1.996201   52.194247  41.033427
4     2.989754    2.007697   77.616376  41.269624


In [15]:
# compute the log likelihood of samples
likelihood_prediction = likelihood_calculator(samples, **target_range)
print(likelihood_prediction.head())

      XLogP      TPSA
0 -1.360093 -2.082480
1 -1.276294 -2.005964
2 -1.128147 -2.165423
3 -1.284850 -1.883462
4 -1.188948 -2.642831


### 4 proposal model

proposal from the given reactant pool, sample(index of reactant) is modified by randomly changing one reactant to a similar one.

In [16]:
# proposal based on samples
new_samples = pool_obj.proposal(samples)
print(samples)
print(new_samples)

['12163.22445', '863.20896', '249087', '153658.23440', '297070', '208421', '412634.601987', '10425.19854', '9361.387984.30667', '50995.305035']
['12163.28507', '22124.20896', '200136', '153658.438826', '491549', '59664', '146642.601987', '28023.19854', '9361.290513.30667', '303868.305035']


### 5 complete run of ismd

In [17]:
# set up initial reactants
cans = [smi for i, smi in enumerate(data['reactant_index'])
        if (data['XLogP'].iloc[i] > 4)]
init_samples = np.random.choice(cans, 10)
print(init_samples)

['3512.448559' '340658' '208158.118041.16794' '167733.537503' '409062'
 '456078.36953' '52646.15893' '14376.18781' '159186.505934' '549477']


In [18]:
# set up annealing schedule
beta = np.hstack([np.linspace(0.01,0.2,20),np.linspace(0.21,0.4,10),np.linspace(0.4,1,10),np.linspace(1,1,10)])
print('Number of steps: %i' % len(beta))
print(beta)

Number of steps: 50
[0.01       0.02       0.03       0.04       0.05       0.06
 0.07       0.08       0.09       0.1        0.11       0.12
 0.13       0.14       0.15       0.16       0.17       0.18
 0.19       0.2        0.21       0.23111111 0.25222222 0.27333333
 0.29444444 0.31555556 0.33666667 0.35777778 0.37888889 0.4
 0.4        0.46666667 0.53333333 0.6        0.66666667 0.73333333
 0.8        0.86666667 0.93333333 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ]


In [19]:
# library for running ismd in XenonPy-iQSPR
from xenonpy.inverse.iqspr import IQSPR

# set up likelihood and modifier models in iQSPR
ismd = IQSPR(estimator=likelihood_calculator, modifier=pool_obj)
    
np.random.seed(201906) # fix the random seed
# main loop of iQSPR
ismd_samples, ismd_loglike, ismd_prob, ismd_freq = [], [], [], []
for s, ll, p, freq in ismd(init_samples, beta, yield_lpf=True):
    ismd_samples.append(s)
    ismd_loglike.append(ll)
    ismd_prob.append(p)
    ismd_freq.append(freq)
# record all outputs
ismd_results = {
    "samples": ismd_samples,
    "loglike": ismd_loglike,
    "prob": ismd_prob,
    "freq": ismd_freq,
    "beta": beta
}


RDKit ERROR: [15:08:05] SMILES Parse Error: extra open parentheses for input: 'CC(C)(C)OC(=O)N1CC[C@@H](Cc2nn(-c3ccc(C(c4ccccc4)(c4ccccc4)n4cccn4)cc3)c(=O)n2-c2ccc(-c3ccc4cccnc4c3)cc2F'
RDKit ERROR: [15:08:05] SMILES Parse Error: extra open parentheses for input: 'CC(C)(C)OC(=O)N1CC[C@@H](Cc2nn(-c3cnn(C(c4ccccc4)(c4ccccc4)c4ccccc4)c3)c(=O)n2-c2ccc(-c3ccc4cccnc4c3)cc2F'
RDKit ERROR: [15:08:17] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 13 14 15 17 18 19
RDKit ERROR: 
RDKit ERROR: [15:08:22] SMILES Parse Error: extra open parentheses for input: 'Cc1cc(-n2c(C[C@@H]3CCN(C(=O)C4CC4)C3)nn(C(=O)C(CCCSC(c3ccccc3)(c3ccccc3)c3ccccc3)CC(=O)OC(C)(C)C)c2=O)c(F'
RDKit ERROR: [15:08:23] SMILES Parse Error: unclosed ring for input: 'Cc1nc(-c2cn(C(c3ccccc3)(c3ccccc3)c3ccccc3)c(=O)n2C[C@@H]2CCN(C(=O)C3CC3)C2)cn1C(c1ccccc1)(c1ccccc1)c1cccc'
RDKit ERROR: [15:08:26] SMILES Parse Error: unclosed ring for input: 'CC(=O)N1CC[C@@H](Cc2nn(C(=O)C(CC(=O)NOC(c3ccccc3)(c3ccccc3)c3ccccc3)CC(=O)OC(C)(C)C)c(=O)

In [20]:
# have a look at the result
ismd_result_df = pd.DataFrame(ismd_results)
ismd_result_df.head()

samples  \
0  [14376.18781, 159186.505934, 167733.537503, 20...   
1  [14376.390089, 159186.30602, 254641.505934, 27...   
2  [160654.395970, 194191, 238605.505934, 3512.17...   
3  [181233, 1901, 238605.265663, 253001.337619, 2...   
4  [147331.337619, 324069.265663, 410297.18781, 4...   

                                             loglike  \
0        XLogP      TPSA
0 -1.449954 -2.420606
1 ...   
1        XLogP      TPSA
0 -1.618544 -2.126752
1 ...   
2        XLogP      TPSA
0 -1.148370 -2.290254
1 ...   
3        XLogP      TPSA
0 -1.111461 -2.096320
1 ...   
4        XLogP      TPSA
0 -0.838483 -2.220596
1 ...   

                                                prob  \
0  [0.09955269694814026, 0.09953953740960717, 0.1...   
1  [0.09961361469186743, 0.09904500956466293, 0.1...   
2  [0.10017170718377574, 0.19991690246637525, 0.1...   
3  [0.10111233182431391, 0.09915898303393672, 0.0...   
4  [0.10156296098223812, 0.0996687845349864, 0.09...   

                             freq  beta  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  0.01  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  0.02  
2     [1, 2, 1, 1, 1, 1, 1, 1, 1]  0.03  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  0.04  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  0.05

In [21]:
ismd_result_df['samples'][0]

array(['14376.18781', '159186.505934', '167733.537503',
       '208158.118041.16794', '340658', '3512.448559', '409062',
       '456078.36953', '52646.15893', '549477'], dtype='<U32')

In [22]:
ismd_result_df['loglike'][0]

XLogP      TPSA
0 -1.449954 -2.420606
1 -1.345152 -2.538628
2 -1.105808 -2.138121
3 -1.356044 -2.385798
4 -0.873934 -2.164831
5 -1.630458 -2.380833
6 -0.947385 -1.893932
7 -0.937115 -2.032380
8 -1.258988 -2.337640
9 -1.172434 -1.861330

In [23]:
ismd_result_df['prob'][0]

array([0.0995527 , 0.09953954, 0.10017848, 0.09968092, 0.10038423,
       0.09941269, 0.10058263, 0.10045379, 0.09982578, 0.10038925])